In [7]:
import sys
print(sys.executable)

/data1/anaconda3/bin/python


In [8]:
import json
# # file_path = './data/WebQuestions.json'
def prepare_dataset(file_path):

    with open(file_path, 'r') as f:
        dataset = json.load(f)
    return dataset

# # dataset = prepare_dataset(file_path)
# # for data in dataset[2:3]:
# #     # print(data)
# #     question = data['RawQuestion']
# #     # print(question)

# def str_match(info, entity_chains):
#     return any(info in entity or entity in info for entity in entity_chains)



In [8]:
## for llama

entity_chain_prompt = """
please identify the entities in the sentence by taking into account the sentence's syntactic structure. 
Below is an example.
sentence: Which place is the madam satan located?
entities: ['place', 'madam satan']
Now answer with the format of the example above.
sentence: %s
entities:"""

prune_prompt = """
Please retrieve relations that are related to any of the target information from the sentence and rate it on a scale of 0 to 10. Be brief and concise.
sentence: Name the president of the country whose main spoken language was Brahui in 1980?
target information: ['president', 'country', 'main spoken language', 'Brahui', '1980']
relations: ['language.human_language.main_country', 'language.human_language.human_language', 'language.human_language.language_family', 'language.human_language.iso_639_3_code', 'base.rosetta.languoid.parent', 'language.human_language.writing_system', 'base.rosetta.languoid.languoid_class'] 
assess: 
- 'language.human_language.main_country' is related to the country, so it's a match, (9) score.
- 'language.human_language.human_language' is kind of related to the main spoken language, so it's a match, (6) score.
- other relations are not related to any information in the sentence, so they are not match, (0) score.
sentence: {}
target information: {}
relations: {}
assess: """

select_node_prompt = """
Please assess the relevance of each candidate entity to the information in the sentence and assign a score ranging from 0 to 10. The format of the candidate entity is (relation, entity).
sentence: Which place is the madam satan located?
information in the sentence: ['place', 'madam satan']
candidate entity: [('film.film.country', 'the USA'), ('film.film.language', 'English')]
assess: 
- ('film.film.country', 'the USA') is related to 'place', so it's a match, (8) score.
- ('film.film.language', 'English')is not related to any information in the sentence,(0) score.
sentence: {}
information in the sentence: {}
candidate entity: {}
assess: """



answer_prompt = """
please answer the question based on the provided triplets. The format of the triplet is (entity1, relation, entity2). If there is not enough information provided in the triplets, answer the question with your own knowledge.
question: Which place is the madam satan located?
triplets: [('madam satan', 'film.film.country', 'the USA'), ('madam satan', 'film.film.language', 'English')]
answer: 'madam satan' is located in 'the USA'.
question: {}
triplets: {}
answer: """


def construct_entity_chain_prompt(question):
    return entity_chain_prompt % question

def construct_select_node_prompt(question, uninvovled_info, candidates):
    return select_node_prompt.format(question, uninvovled_info, candidates)


def construct_answer_prompt(question, routes):
    return answer_prompt.format(question, routes)

def construct_prune_prompt(question, aim_info, related_node):
    return prune_prompt.format(question, aim_info, related_node)


In [ ]:
from transformers import AutoTokenizer, AutoModelForCausalLM
import torch

model_id = "/models/Llama3-8B-Instruct/"
tokenizer = AutoTokenizer.from_pretrained(model_id)
model = AutoModelForCausalLM.from_pretrained(
    model_id,
    torch_dtype=torch.bfloat16,
    device_map="auto",
)
model=model.eval()

In [7]:
def chat_llama(prompt):
    messages = [
        {"role": "system", "content": "You are an AI assistant that helps people find information."},
        {"role": "user", "content": prompt},
    ]

    input_ids = tokenizer.apply_chat_template(
        messages,
        add_generation_prompt=True,
        return_tensors="pt"
    ).to(model.device)

    terminators = [
        tokenizer.eos_token_id,
        tokenizer.convert_tokens_to_ids("<|eot_id|>")
    ]

    outputs = model.generate(
        input_ids,
        max_new_tokens=512,
        eos_token_id=terminators,
        do_sample=False,
        # temperature=0.6,
        top_p=0.9,
    )
    response = outputs[0][input_ids.shape[-1]:]
    return tokenizer.decode(response, skip_special_tokens=True)

#prompt = """Who are you?"""
#print(chat_llama(prompt))


In [ ]:
from openai import OpenAI


def chat_gpt(prompt, model="gpt-3.5-turbo-16k"):
    client = OpenAI(
        base_url="",
        api_key=""
    )

 
    messages=[
            {"role": "system", "content": "You are a helpful assistant."},
            {"role": "user", "content": prompt}
        ]

    completion = client.chat.completions.create(
        model=model,
        messages=messages,
        temperature=0
    )

    # 非流式输出获取结果
    return completion.choices[0].message.content


In [5]:
import re
import json
import ast

# use_model = "gpt-3.5-turbo-16k"
# use_model = "gpt-4"
# use_model = "glm-4-9b-chat"
use_model = "llama"

response_path = 'llama_prompt_response.json'
# response_path = 'chatgpt_prompt_response.json'
#response_path = 'gpt4_prompt_response.json'
# response_path = 'glm4_9b_chat_prompt_response.json'

def extract_entity_chain(question, model=use_model):
    prompt = construct_entity_chain_prompt(question)
    if use_model == "llama":
        response = chat_llama(prompt)
    elif use_model == "gpt-3.5-turbo-16k":
        response = chat_gpt(prompt, model="gpt-3.5-turbo-16k")
    elif use_model == "gpt-4":
        response = chat_gpt(prompt, model="gpt-4")
    elif use_model == "glm-4-9b-chat":
        response = glm4_9b_chat(prompt)

    response = response.split('entities: ')[-1]

    response = response.split('<|user|>')[0]

    tmp = {}
    tmp['prompt'] = [question]
    tmp['response'] = response
    with open(response_path, 'a', encoding='utf-8') as file:
        file.write('extract_clues:  ')
        json.dump(tmp, file, ensure_ascii=False)
        file.write('\n')
    # print(response)
    new_list = ast.literal_eval(response)
    # print(new_list)
    return new_list
    


In [9]:


def relation_mapping(question, aim_info, entity, related_node, model=use_model):
    prompt = construct_prune_prompt(question, aim_info, related_node)
    if use_model == "llama":
        response = chat_llama(prompt)
    elif use_model == "gpt-3.5-turbo-16k":
        response = chat_gpt(prompt, model="gpt-3.5-turbo-16k")
    elif use_model == "gpt-4":
        response = chat_gpt(prompt, model="gpt-4")
    elif use_model == "glm-4-9b-chat":
        response = glm4_9b_chat(prompt)
    # print('relation_mapping: ')
    # print(response)
    response = response.split('assess:')[-1]
    response = response.split('<|user|>')[0]
    tmp = {}
    tmp['prompt'] = [question, aim_info, entity, related_node]
    tmp['response'] = response
    with open(response_path, 'a', encoding='utf-8') as file:
        file.write('relation_mapping:  ')
        json.dump(tmp, file, ensure_ascii=False)
        file.write('\n')
    result = []
    for line in response.split("\n"):
        relation = []
        for info in related_node:
            if str(info) in line:
                relation.append(info)
        if len(relation) != 1:
            continue
        clues = []
        for info in aim_info:
            if info in line:
                clues.append(info)
        seg = line.split(',')
        if 'score' in seg[-1]:
            scores = re.findall(r'\d+', seg[-1])
        else:
            scores = []
        if scores == []:
            continue
        print(relation, clues, scores)
        if clues == [] or int(scores[0]) < 5:
            continue
        for clue in clues:
            result.append((relation[0], clue, scores[0]))
        print(result)
        if len(result) > 1:
            result.sort(key=lambda x: int(x[2]), reverse=True)
            result = result
    return result

# q = "what county is st. louis park in?" 
# aim_info = ['county', 'st. louis park'] 
# entity = ('county', 'st. louis park') 
# rn = ['common.topic.article', 'common.topic.description', 'common.topic.image', 'common.topic.webpage', 'user.avh.default_domain.ellerdale_topic.ellerdale_id','base.ontologies.ontology_instance.equivalent_instances','common.image.appears_in_topic_gallery','base.ontologies.ontology_instance_mapping.freebase_topic']
# print(prune_node(q, aim_info, entity, rn))

def entity_mapping(question, uninvovled_info, candidates, model=use_model):
    prompt = construct_select_node_prompt(question, uninvovled_info, candidates)
    if use_model == "llama":
        response = chat_llama(prompt)
    elif use_model == "gpt-3.5-turbo-16k":
        response = chat_gpt(prompt, model="gpt-3.5-turbo-16k")
    elif use_model == "gpt-4":
        response = chat_gpt(prompt, model="gpt-4")
    elif use_model == "glm-4-9b-chat":
        response = glm4_9b_chat(prompt)
    # print('entity_mapping:')
    # print(response)
    # response = """- ('county', ' hennepin county, minnesota') is related to 'county', so it's a match, (9) score.\n- ('location.location.containedby', 'chicago, milwaukee, st. paul and pacific depot (st. louis park, minnesota)') is related to 'location.location.containedby', 'st. louis park', so it's a match, (8) score.\n- ('location.location.containedby', 'University of Phoenix-Minneapolis/St Paul Campus') is not related to any information in the sentence,(0) score.\n- ('location.location.containedby', 'Toby Keith's I Love This Bar & Grill') is not related to any information in the sentence,(0) score.\n- ('location.location.containedby', 'DeVry University-Minnesota') is not related to any information in the sentence,(0) score."""
    response = response.split('assess:')[-1]
    response = response.split('<|user|>')[0]
    tmp = {}
    tmp['prompt'] = [question, uninvovled_info, candidates]
    tmp['response'] = response
    with open(response_path, 'a', encoding='utf-8') as file:
        file.write('entity_mapping:  ')
        json.dump(tmp, file, ensure_ascii=False)
        file.write('\n')

    result = []

    for line in response.split("\n"):
        relation = []
        for info in candidates:
            if str(info) in line:
                relation.append(info)
        if len(relation) != 1:
            continue
        clues = []
        for info in uninvovled_info:
            if info in line:
                clues.append(info)
        seg = line.split(',')
        if 'score' in seg[-1]:
            scores = re.findall(r'\d+', seg[-1])
        else:
            scores = []
        if scores == []:
            continue
        if clues == [] or scores[0] == '0':
            continue
        for clue in clues:
            if clue not in uninvovled_info:
                continue
            result.append((relation[0], clue, scores[0]))
    return result
# c = [["location.location.containedby", "united states"], ["location.location.containedby", "minnesota"], ["location.location.containedby", "hennepin county, minnesota"], ["location.location.containedby", "minneapolis – saint paul"], ["location.location.containedby", "Toby Keith's I Love This Bar & Grill"], ["location.location.containedby", "University of Phoenix-Minneapolis/St Paul Campus"], ["location.location.containedby", "chicago, milwaukee, st. paul and pacific depot (st. louis park, minnesota)"], ["location.location.containedby", "DeVry University-Minnesota"]]
# q = "what county is st. louis park in?"
# ui = ['county'] 
# # c = [('location.location.containedby', 'italy'), ('location.location.containedby', 'province of cremona'), ('location.location.containedby', 'lombardy')]
# print(select_node(q, ui, c))

def answer_question(question, routes, model=use_model):
    s = ''
    for r in routes:
        s += str(r) + ', '
    prompt = construct_answer_prompt(question, s)
    if use_model == "llama":
        response = chat_llama(prompt)
    elif use_model == "gpt-3.5-turbo-16k":
        response = chat_gpt(prompt, model="gpt-3.5-turbo-16k")
    elif use_model == "gpt-4":
        response = chat_gpt(prompt, model="gpt-4")
    elif use_model == "glm-4-9b-chat":
        response = glm4_9b_chat(prompt)
    print('answer:')
    print(response)
    response = response.replace(prompt, '').split('sentence:')[0].strip()
    response = response.split('Please let me know')[0].strip('\n')
    response = response.split('<|user|>')[0]
    tmp = {}
    tmp['prompt'] = prompt
    tmp['response'] = response
    with open(response_path, 'a', encoding='utf-8') as file:
        file.write('answer:  ')
        json.dump(tmp, file, ensure_ascii=False)
        file.write('\n')
    return response

# print(answer_question("where did drew brees go to college wikianswers?",[]))


prune_node

- 'common.topic.article' is related to 'st. louis park', but not directly to the county, (3) score.
- 'common.topic.description' is related to 'st. louis park', but not directly to the county, (3) score.
- 'base.ontologies.ontology_instance.equivalent_instances' could potentially be related to 'st. louis park' but not specifically to the county, (3) score.
- Other relations are not directly related to the county, (0) score. <|user|>
[('common.topic.article', '3'), ('st. louis park', '3'), ('common.topic.description', '3'), ('st. louis park', '3'), ('base.ontologies.ontology_instance.equivalent_instances', '3'), ('st. louis park', '3')]
[('common.topic.article', '3'), ('st. louis park', '3'), ('common.topic.description', '3'), ('st. louis park', '3'), ('base.ontologies.ontology_instance.equivalent_instances', '3'), ('st. louis park', '3')]


In [4]:

import re

name_dict = {}

def read_MapName():
    map_list = []
    with open("./mid2name.txt", "r", encoding="utf-8") as file:
        lines = file.readlines()
        for line in lines:
            try:
                map_list = re.split(r"\t", line)
                map_list[0] = map_list[0].replace("/", ".")[1:]
                map_list[1] = map_list[1].strip("\n").lower()
                if map_list[0] in name_dict:
                    v = name_dict[map_list[0]]
                    v.append(map_list[1])
                    name_dict[map_list[0]] = v
                else:
                    name_dict[map_list[0]] = [map_list[1]]
                if map_list[1] in name_dict:
                    v = name_dict[map_list[1]]
                    v.append(map_list[0])
                    name_dict[map_list[1]] = v
                else:
                    name_dict[map_list[1]] = [map_list[0]]
            except Exception as e:
                print(f"The row {map_list} has error!!!")
                print(e)
                return

read_MapName()

In [5]:
def query_node_by_id(id):
    if id in name_dict:
        nodes = []
        for name in name_dict[id]:
            nodes.append(('', name, id))
        return nodes
    else:
        return None
    
def query_node_by_name(name):
    name = name.lower()
    if name in name_dict:
        nodes = []
        for id in name_dict[name]:
            nodes.append(('', name, id))
        return nodes
    else:
        return None

import requests
from SPARQLWrapper import SPARQLWrapper, JSON
def get_wikidata_name(id):
    url = "http://www.wikidata.org/w/api.php"
    params = {
        "action": "wbgetentities",
        "ids": id,
        "format": "json",
        "props": "labels",
        "languages": "en"
    }

    response = requests.get(url, params=params)
    data = response.json()

    # Get the label in English
    label = data['entities'][id]['labels']['en']['value']
    return label

# from TOG freebase_func
SPARQLPATH = ""

def get_id_from_base(entity_id):
    sparql_id = """select DISTINCT ?e where {<id> <http://rdf.freebase.com/ns/type.object.name> ?e.}
            """.replace("id", "http://rdf.freebase.com/ns/" + entity_id)
    sparql = SPARQLWrapper(SPARQLPATH)
    sparql.setQuery(sparql_id)
    sparql.setReturnFormat(JSON)
    results = sparql.query().convert()
    if len(results["results"]["bindings"]) > 0:
        return results["results"]["bindings"][0]['e']['value']
    
    sparql_id = """select DISTINCT ?e where {<id> <http://www.w3.org/2002/07/owl#sameAs> ?e.}
            """.replace("id", "http://rdf.freebase.com/ns/" + entity_id)
    sparql = SPARQLWrapper(SPARQLPATH)
    sparql.setQuery(sparql_id)
    sparql.setReturnFormat(JSON)
    results = sparql.query().convert()
    if len(results["results"]["bindings"]) > 0:
        id = results["results"]["bindings"][0]['e']['value']
        id = id.split('/')[-1]
        return get_wikidata_name(id)
    return "UnName_Entity"


# print(query_node_by_name("romney"))

# print(query_node_by_id('m.015_hb'))
# print(get_id_from_base('m.015_hb'))
# Count unique entities


None


In [ ]:
# !pip install SPARQLWrapper
from SPARQLWrapper import SPARQLWrapper, JSON
import re

SPARQLPATH = ""

def get_name_by_id(id):
    if id in name_dict:
        return name_dict[id][0]
    return get_id_from_base(id)

# print(get_name_by_id('g.125g4ph1r'))

def get_directed_related_nodes_by_id(id, tail = True):   # 根据一个节点查询所有其他所有的关系和对应的尾节点
    result = []
    sparql = SPARQLWrapper(SPARQLPATH)
    #print('before')
    if tail:
        sparql_txt = """select DISTINCT ?r ?e where {<id> ?r ?e.} limit 1000
            """.replace("id", "http://rdf.freebase.com/ns/" + id)
    else:
        sparql_txt = """select DISTINCT ?r ?e where {?e ?r <id>.} limit 1000
            """.replace("id", "http://rdf.freebase.com/ns/" + id)
    sparql.setQuery(sparql_txt)
    sparql.setReturnFormat(JSON)
    results = sparql.query().convert()
    #print('here')
    # for i in results['results']['bindings']:
    #     print(i)
    try:
        for item in results['results']['bindings']:
            node_id = ''
            if item['r']['value'].split('/')[-1] in ['type.object.name', 'owl#sameAs']:
                continue
            if item['e']['type'] == 'uri':
                node_id = item['e']['value'].split('/')[-1]
                if node_id[0] != 'm':
                    continue
                name = get_name_by_id(node_id)
                
            else:
                name = item["e"]["value"]
            r = item["r"]["value"].replace("http://rdf.freebase.com/ns/", "")
            result.append((r, name, node_id, r))
    except Exception as e:
        print(e)
    return result

def get_related_nodes_by_id(id):
    results = get_directed_related_nodes_by_id(id, True)
    results.extend(get_directed_related_nodes_by_id(id, False))
    return results

# print(get_directed_related_nodes_by_id('m.078w2'))
# print(get_related_nodes_by_id('m.07qymj'))
